In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import ibapi
from ibapi.client import EClient
from ibapi.wrapper import EWrapper

In [5]:
class StockTradingBot:
    def __init__(self, symbol, short_window, long_window, initial_cash):
        self.symbol = symbol
        self.short_window = short_window
        self.long_window = long_window
        self.cash = initial_cash
        self.stock_balance = 0
        self.history = []

    def get_stock_data(self, start_date, end_date):
        data = yf.download(self.symbol, start=start_date, end=end_date)
        return data

    def calculate_sma(self, data, window):
        return data['Close'].rolling(window=window).mean()

    def buy(self, price, amount):
        total_cost = price * amount
        if self.cash >= total_cost:
            self.cash -= total_cost
            self.stock_balance += amount
            self.history.append(f"Bought {amount} shares at ${price:.2f} each")

    def sell(self, price, amount):
        if self.stock_balance >= amount:
            total_sale = price * amount
            self.cash += total_sale
            self.stock_balance -= amount
            self.history.append(f"Sold {amount} shares at ${price:.2f} each")

    def execute_strategy(self, data):
        short_sma = self.calculate_sma(data, self.short_window)
        long_sma = self.calculate_sma(data, self.long_window)

        for i in range(self.long_window, len(data)):
            if short_sma.iloc[i] > long_sma.iloc[i]:  # Buy signal
                self.buy(data['Close'].iloc[i], 10)  # Example: Buy 10 shares
            elif short_sma.iloc[i] < long_sma.iloc[i]:  # Sell signal
                self.sell(data['Close'].iloc[i], 10)  # Example: Sell 10 shares

    def run(self):
        data = self.get_stock_data("2022-01-01", "2023-01-01")  # Adjust date range as needed
        self.execute_strategy(data)
        self.display_portfolio(data)

    def display_portfolio(self, data):
        print("Portfolio Summary:")
        print(f"Cash: ${self.cash:.2f}")
        print(f"Stock Balance: {self.stock_balance} shares")
        print(f"Portfolio Value: ${(self.cash + self.stock_balance * data['Close'].iloc[-1]):.2f}")

if __name__ == "__main__":
    bot = StockTradingBot(symbol="AAPL", short_window=50, long_window=200, initial_cash=10000)
    bot.run()

[*********************100%***********************]  1 of 1 completed

Portfolio Summary:
Cash: $10000.00
Stock Balance: 0 shares
Portfolio Value: $10000.00


In [11]:
class IBApi(EWrapper, EClient):

    def __init__(self):
        EClient.__init__(self, self)

class Bot:
    ib = None
    def __init__(self):
        ib = IBApi()

bot = Bot()